## Q1

### a)

In [1]:
# 1. Implement in python program of the following problems using Bayes Theorem.
# a) Of the students in the college, 60% of the students reside in the hostel and 40% of the students are day
# scholars. Previous year results report that 30% of all students who stay in the hostel scored A Grade and 20%
# of day scholars scored A grade. At the end of the year, one student is chosen at random and found that he/she
# has an A grade. What is the probability that the student is a hosteler?

In [2]:
P_H = 0.6
P_D = 0.4
P_A_given_H = 0.3
P_A_given_D = 0.2
# find probability that the student is a hosteler given that he/she has grade A
# i.e. find P_H_given_A
# Law of total probability
P_A = P_A_given_H * P_H + P_A_given_D * P_D
print("Evidence: ",P_A)
P_H_given_A = (P_A_given_H * P_H)/P_A
print("Required(posterior) probability: ",P_H_given_A)


Evidence:  0.26
Required(posterior) probability:  0.6923076923076923


### b)

In [3]:
# b) Suppose you're testing for a rare disease, and you have the following information:
# The disease has a prevalence of 0.01 (1% of the population has the disease).
# The test is not perfect:
# The test correctly identifies the disease (true positive) 99% of the time (sensitivity).
# The test incorrectly indicates the disease (false positive) 2% of the time (1 - specificity).
# Calculate the probability of having the disease given a positive test result using Bayes' theorem

In [4]:
P_disease = 0.01
P_no_disease = 1-0.01
TP = 0.99 # TP=P_positive_given_disease 
FP = 0.02
# find P_disease_given_positive
P_positive = (P_disease * TP) + (P_no_disease * FP)
print("Total positive cases: ",P_positive)
P_disease_given_positive = (TP * P_disease)/P_positive
print("Required (posterior) probability: ",P_disease_given_positive)

Total positive cases:  0.029700000000000004
Required (posterior) probability:  0.3333333333333333


## Q2

In [5]:
# 2. Develop a function python code for Naïve Bayes classifier from scratch without using scikit-learn library,
# to predict whether the buyer should buy computer or not. Consider a following sample training dataset stored
# in a CSV file containing information about following buyer conditions (such as “<=30,” “medium,” “Yes,”
# and “fair”) and whether the player played golf (“Yes” or “No”).


In [6]:
import numpy as np
import pandas as pd

df = pd.read_csv('comp_buyer.csv')
df.head()

,age,income,student,credit_rating,computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31...40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes


In [7]:
# to find: P(yes|"<=30","medium","student","fair")
# P(yes|"<=30","medium","yes","fair") = P("<=30"|yes)*P("medium"|yes)*P("student"|yes)*P("fair"|yes)*P(yes)

In [8]:
# manual solution(intuitively)

age_df = pd.crosstab(df['age'],df['computer'])
age_df

computer,no,yes
age,,
31...40,0,4
<=30,3,2
>40,2,3


In [9]:
P_ReqAge_given_yes = 2/9

In [10]:
income_df = pd.crosstab(df['income'],df['computer'])
income_df

computer,no,yes
income,,
high,2,2
low,1,3
medium,2,4


In [11]:
P_ReqIncome_given_yes = 4/9

In [12]:
student_df = pd.crosstab(df['student'],df['computer'])
student_df

computer,no,yes
student,,
no,4,3
yes,1,6


In [13]:
P_ReqStudent_given_yes = 6/9

In [14]:
credit_rating_df = pd.crosstab(df['credit_rating'],df['computer'])
credit_rating_df

computer,no,yes
credit_rating,,
excellent,3,3
fair,2,6


In [15]:
P_ReqCreditRating_given_yes = 6/9

In [16]:
P_yes = df['computer'].value_counts('yes')
P_yes

computer
yes    0.642857
no     0.357143
Name: proportion, dtype: float64

In [17]:
9/14

0.6428571428571429

In [18]:
P_required_yes = P_ReqAge_given_yes*P_ReqIncome_given_yes*P_ReqStudent_given_yes*P_ReqCreditRating_given_yes*P_yes
P_required_yes

computer
yes    0.028219
no     0.015677
Name: proportion, dtype: float64

In [19]:
# P_no = df['computer'].value_counts('no')
P_no = df['computer'].value_counts()/df.shape[0]

P_no

computer
yes    0.642857
no     0.357143
Name: count, dtype: float64

In [20]:
P_required_no = (1-P_ReqAge_given_yes)*(1-P_ReqIncome_given_yes)*(1-P_ReqStudent_given_yes)*(1-P_ReqCreditRating_given_yes)*(P_no)
P_required_no

computer
yes    0.030864
no     0.017147
Name: count, dtype: float64

In [21]:
import numpy as np

# Function to calculate the prior probability
def prior_probability(df, target_col, target_val):
    total_samples = df.shape[0]
    target_count = df[df[target_col] == target_val].shape[0]
    return target_count / total_samples

# Function to calculate the conditional probability
def conditional_probability(df, feature_col, feature_val, target_col, target_val):
    target_df = df[df[target_col] == target_val]
    feature_count = target_df[target_df[feature_col] == feature_val].shape[0]
    total_target = target_df.shape[0]
    if total_target == 0:
        return 0
    return feature_count / total_target

# Naive Bayes probability calculation
def naive_bayes(df, conditions, target_col, target_val):
    # Calculate prior probability P(target)
    prior_prob = prior_probability(df, target_col, target_val)
    
    # Calculate likelihood P(conditions | target)
    likelihood = 1
    for feature, value in conditions.items():
        likelihood *= conditional_probability(df, feature, value, target_col, target_val)
    
    # Calculate the total probability
    return prior_prob * likelihood

# Define conditions: Age <= 30, Income = medium, Student = yes, Credit Rating = fair
conditions = {
    "age": "<=30",
    "income": "medium",
    "student": "yes",
    "credit_rating": "fair"
}

# Calculate the probability of buying a computer given the conditions
probability_yes = naive_bayes(df, conditions, "computer", "yes")
probability_no = naive_bayes(df, conditions, "computer", "no")

# Normalize the probabilities (for comparison)
total_prob = probability_yes + probability_no
normalized_yes = probability_yes / total_prob if total_prob != 0 else 0
normalized_no = probability_no / total_prob if total_prob != 0 else 0

normalized_yes, normalized_no

probability_yes, probability_no


(0.028218694885361547, 0.006857142857142858)

## Q3

In [22]:
# 3. Write a Python function to implement the Naive Bayes classifier without using the scikit-learn library for the
# following sample training dataset stored as a .CSV file. Calculate the accuracy, precision, and recall for your train/test
# dataset.
# a. Build a classifier that determines whether a text is about sports or not.
# b. Determine which tag the sentence "A very close game" belongs to.

In [23]:
df = pd.read_csv('TextClassifier.csv')
df.head()

,Text,Tag
0,a great game,Sports
1,the election was over,Not Sports
2,very clean match,Sports
3,a clean but forgettable game,Sports
4,it was a close election,Not Sports


In [37]:
class NaiveBayesTextClassifier:
    def __init__(self, df, s_tag='Sports', ns_tag='Not Sports'):
        self.df = df
        self.x = df['Text'].values
        self.y = df['Tag'].values
        self.s_tag = s_tag
        self.ns_tag = ns_tag
        self.tot = 14  # Adjust this value as needed for Laplace smoothing
        self.s_word_count = 0
        self.ns_word_count = 0
        self.s_count = 0
        self.ns_count = 0
        self.calculate_word_counts()
        self.calculate_class_counts()

    def calculate_word_counts(self):
        # Calculate total word counts for 'S' and 'NS'
        for i, j in zip(self.x, self.y):
            # w = i.split()
            if j == self.s_tag:
                w = i.split()
                self.s_word_count += len(w)
            else:
                w = i.split()
                self.ns_word_count += len(w)
        print("Word count in Sports: ",self.s_word_count)
        print("Word count in Not Sports: ",self.ns_word_count)
        

    def calculate_class_counts(self):
        # Count the occurrences of 'S' and 'NS' tags
        for label in self.y:
            if label == self.s_tag:
                self.s_count += 1
            else:
                self.ns_count += 1
        print("Count of Sports tag: ",self.s_count)
        print("Count of Not Sports tag: ",self.ns_count)
        

    def find_word_s(self, word):
        # Find the probability of a word given class 'S'
        count = 0
        for i, j in zip(self.x, self.y):
            w = i.split()
            if j == self.s_tag and word in w:
                count += 1

        print(f"P(word='{word}'|Sports): ",(count + 1) / (self.s_word_count + self.tot))
        return (count + 1) / (self.s_word_count + self.tot)  # Laplace smoothing
        
        
        # if count==0:
        #     return 1/(self.s_word_count+self.tot)
        # else:
        #     return (count+1)/(self.s_word_count+self.tot)
        # print("P(word|Sports): ",(count + 1) / (self.s_word_count + self.tot))
        # return (count + 1) / (self.s_word_count + self.tot)  # Laplace smoothing

    def find_word_ns(self, word):
        # Find the probability of a word given class 'NS'
        count = 0
        for i, j in zip(self.x, self.y):
            w = i.split()
            if j == self.ns_tag and word in w:
                count += 1
                
        print(f"P(word='{word}'|Not Sports): ",(count + 1) / (self.ns_word_count + self.tot))
        return (count + 1) / (self.ns_word_count + self.tot)  # Laplace smoothing
        
         
        # if count==0:
        #     return 1/(self.ns_word_count+self.tot)
        # else:
        #     return (count+1)/(self.ns_word_count+self.tot)
        # print("P(word|Not Sports): ",(count + 1) / (self.ns_word_count + self.tot))
        # return (count + 1) / (self.ns_word_count + self.tot)  # Laplace smoothing

    def classify(self, sentence):
        sent = sentence.split()
        
        # Calculate P(word|S) and P(word|NS)
        p_word_s = [self.find_word_s(i) for i in sent]
        p_word_ns = [self.find_word_ns(i) for i in sent]
        print("P(word|Sports): ",p_word_s)
        print("P(word|Not Sports): ",p_word_ns)
        
        
        # Prior probabilities
        p_s = self.s_count / len(self.y)
        p_ns = self.ns_count / len(self.y)
        
        # Calculate P(sentence|S) and P(sentence|NS)
        p_text_s = 1
        for prob in p_word_s:
            p_text_s *= prob
        p_s_text = p_text_s * p_s
        print(p_s_text)
        
        p_text_ns = 1
        for prob in p_word_ns:
            p_text_ns *= prob
        p_ns_text = p_text_ns * p_ns
        print(p_ns_text)
        
        
        # Classification decision
        if p_s_text >= p_ns_text:
            return "Sports"
        else:
            return "Not Sports"

    def evaluate(self):
        # Initialize counters for TP, FP, FN, and TN
        tp, fp, fn, tn = 0, 0, 0, 0
        predictions = []

        # Predict for each entry in the dataset and compare with actual label
        for i, actual in enumerate(self.y):
            # predicted = self.classify(self.x[i])
            # predictions.append(predicted)
            sentence = self.df['Text'].iloc[i]  # Correct way to access the sentence
            # sentence = 'a very close game'
            predicted = self.classify(sentence)

            # True Positive (TP): Correctly predicted Sports
            if actual == self.s_tag and predicted == self.s_tag:
                tp += 1

            # True Negative (TN): Correctly predicted Not Sports
            elif actual == self.ns_tag and predicted == self.ns_tag:
                tn += 1

            # False Positive (FP): Incorrectly predicted Sports
            elif actual == self.ns_tag and predicted == self.s_tag:
                fp += 1

            # False Negative (FN): Incorrectly predicted Not Sports
            elif actual == self.s_tag and predicted == self.ns_tag:
                fn += 1

        # Calculate accuracy, precision, recall
        accuracy = (tp + tn) / len(self.y)
        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall
        }
# Example usage:
# Assuming 'df' is a pandas DataFrame with columns 'text' and 'tag'
# df = pd.DataFrame({
#     'text': ['A great game', 'The election was over', 'Very clean match', 'A clean but forgettable game', 'It was a close election'],
#     'tag': ['S', 'NS', 'S', 'S', 'NS']
# })
df = pd.read_csv('TextClassifier.csv')

# Instantiate the classifier and classify a sentence
classifier = NaiveBayesTextClassifier(df)
sentence = 'a very close game'
classification = classifier.classify(sentence)

print(f'The sentence "{sentence}" is classified as: {classification}')

metrics = classifier.evaluate()
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")


Word count in Sports:  11
Word count in Not Sports:  9
Count of Sports tag:  3
Count of Not Sports tag:  2
P(word='a'|Sports):  0.12
P(word='very'|Sports):  0.08
P(word='close'|Sports):  0.04
P(word='game'|Sports):  0.12
P(word='a'|Not Sports):  0.08695652173913043
P(word='very'|Not Sports):  0.043478260869565216
P(word='close'|Not Sports):  0.08695652173913043
P(word='game'|Not Sports):  0.043478260869565216
P(word|Sports):  [0.12, 0.08, 0.04, 0.12]
P(word|Not Sports):  [0.08695652173913043, 0.043478260869565216, 0.08695652173913043, 0.043478260869565216]
2.7647999999999993e-05
5.7175324559303314e-06
The sentence "a very close game" is classified as: Sports
P(word='a'|Sports):  0.12
P(word='great'|Sports):  0.08
P(word='game'|Sports):  0.12
P(word='a'|Not Sports):  0.08695652173913043
P(word='great'|Not Sports):  0.043478260869565216
P(word='game'|Not Sports):  0.043478260869565216
P(word|Sports):  [0.12, 0.08, 0.12]
P(word|Not Sports):  [0.08695652173913043, 0.043478260869565216, 0.0

## Aditional Questions

## Q1

In [25]:
# 1. Write a function python program to implement the naïve Bayesian classifier without using scikit-learn
# library for the following sample training data set stored as a .CSV file. Calculate the accuracy, precision, and
# recall for your train/test data set. To classify ‘If the weather is sunny, then the Player should play or not’?

In [26]:
# To find P(yes|sunny) =(P(sunny|yes)*P(yes))/P(sunny)
play = pd.read_csv('play2.csv')
play.head()

,Outlook,Play
0,Rainy,Yes
1,Sunny,Yes
2,Overcast,Yes
3,Overcast,Yes
4,Sunny,No


In [27]:
import numpy as np

# Function to calculate the prior probability
def prior_probability(df, target_col, target_val):
    total_samples = df.shape[0]
    target_count = df[df[target_col] == target_val].shape[0]
    return target_count / total_samples

# Function to calculate the conditional probability
def conditional_probability(df, feature_col, feature_val, target_col, target_val):
    target_df = df[df[target_col] == target_val]
    feature_count = target_df[target_df[feature_col] == feature_val].shape[0]
    total_target = target_df.shape[0]
    if total_target == 0:
        return 0
    return feature_count / total_target
    # unique_values = df[feature_col].nunique()
    
    # if total_target == 0:
    #     return 0
    # return (feature_count+1) / (total_target+unique_values)  #Laplace smoothing

# Naive Bayes probability calculation
def naive_bayes(df, conditions, target_col, target_val):
    # Calculate prior probability P(target)
    prior_prob = prior_probability(df, target_col, target_val)
    
    # Calculate likelihood P(conditions | target)
    likelihood = 1
    for feature, value in conditions.items():
        likelihood *= conditional_probability(df, feature, value, target_col, target_val)
    
    # Calculate the total probability
    return prior_prob * likelihood

conditions = {
    "Outlook":"Sunny"
}

# Calculate the probability of buying a computer given the conditions
probability_yes = naive_bayes(play, conditions, "Play", "Yes")
probability_no = naive_bayes(play, conditions, "Play", "No")

# Normalize the probabilities (for comparison)
total_prob = probability_yes + probability_no
normalized_yes = probability_yes / total_prob if total_prob != 0 else 0
normalized_no = probability_no / total_prob if total_prob != 0 else 0

normalized_yes, normalized_no

print("P(yes|sunny): ",probability_yes)
print("P(no|sunny): ",probability_no)

if (probability_yes>=probability_no):
    print("Player should play! :)")
else:
    print("Player should not play! :(")

# answer matches manual solution yayyy!

P(yes|sunny):  0.21428571428571427
P(no|sunny):  0.14285714285714285
Player should play! :)


## Q2

In [28]:
# 2.Develop a function python code for Naïve Bayes classifier from scratch without using scikit-learn library,
# to predict whether the player should play golf based on weather conditions. Consider a following sample
# training dataset stored in a CSV file containing information about following weather conditions (such as
# “Sunny,” “Hot,” “High Humidity,” and “Not Windy”) and whether the player played golf (“Yes” or “No”).

In [29]:
PlayGolf = pd.read_csv('PlayGolf.csv')
PlayGolf.head()

,Weather,Temperature,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes


In [30]:
import numpy as np

# Function to calculate the prior probability
def prior_probability(df, target_col, target_val):
    total_samples = df.shape[0]
    target_count = df[df[target_col] == target_val].shape[0]
    return target_count / total_samples

# Function to calculate the conditional probability
def conditional_probability(df, feature_col, feature_val, target_col, target_val):
    target_df = df[df[target_col] == target_val]
    feature_count = target_df[target_df[feature_col] == feature_val].shape[0]
    total_target = target_df.shape[0]

    unique_values = df[feature_col].nunique()
    
    if total_target == 0:
        return 0
    return (feature_count+1) / (total_target+unique_values)  #Laplace smoothing

# Naive Bayes probability calculation
def naive_bayes(df, conditions, target_col, target_val):
    # Calculate prior probability P(target)
    prior_prob = prior_probability(df, target_col, target_val)
    
    # Calculate likelihood P(conditions | target)
    likelihood = 1
    for feature, value in conditions.items():
        likelihood *= conditional_probability(df, feature, value, target_col, target_val)
    
    # Calculate the total probability
    return prior_prob * likelihood

conditions = {
    "Weather": "Sunny",
    "Temperature": "Hot",
    "Humidity": "High",
    "Windy": "False"
}

# Calculate the probability of buying a computer given the conditions
probability_yes = naive_bayes(PlayGolf, conditions, "Play Golf", "Yes")
probability_no = naive_bayes(PlayGolf, conditions, "Play Golf", "No")

# Normalize the probabilities (for comparison)
total_prob = probability_yes + probability_no
normalized_yes = probability_yes / total_prob if total_prob != 0 else 0
normalized_no = probability_no / total_prob if total_prob != 0 else 0

normalized_yes, normalized_no

print("P(yes|conditions): ",probability_yes)
print("P(no|conditions): ",probability_no)

if (probability_yes>=probability_no):
    print("Player should play golf! :)")
else:
    print("Player should not play golf! :(")


P(yes|conditions):  0.001770956316410862
P(no|conditions):  0.00512481778425656
Player should not play golf! :(


## Q3

In [31]:
# 3. Write a function python program to implement the naïve Bayesian classifier without using scikit-learn
# library for a following sample training data set stored as a.CSV file. Calculate the accuracy, precision, and
# recall for your train/test data set.

In [32]:
text = pd.read_csv('TextClassifier2.csv')
text.head()

,Text Documents,Label
0,i love this sandwich,pos
1,this is an amazing place,pos
2,i feel very good about these beers,pos
3,this is my best work,pos
4,what an awesome view,pos


In [33]:
def count_unique_words(df):
    # Initialize an empty set to store unique words
    unique_words = set()

    # Iterate over each text document
    for text in df['Text Documents']:
        # Split the text into words
        words = text.split()

        # Add each word to the set of unique words
        unique_words.update(words)

    # Return the total number of unique words
    return len(unique_words)

# Load the data
df = pd.read_csv('TextClassifier2.csv')

# Calculate the total number of unique words
num_unique_words = count_unique_words(df)

print(f'Total number of unique words: {num_unique_words}')

Total number of unique words: 59


In [34]:
class NaiveBayesTextClassifier:
    def __init__(self, df, pos_tag='pos', neg_tag='neg'):
        self.df = df
        self.x = df['Text Documents'].values
        self.y = df['Label'].values
        self.pos_tag = pos_tag
        self.neg_tag = neg_tag
        self.tot = 59  # Adjust this value as needed for Laplace smoothing
        self.pos_word_count = 0
        self.neg_word_count = 0
        self.pos_count = 0
        self.neg_count = 0
        self.calculate_word_counts()
        self.calculate_class_counts()

    def calculate_word_counts(self):
        # Calculate total word counts for 'pos' and 'neg'
        for i, j in zip(self.x, self.y):
            # w = i.split()
            if j == self.pos_tag:
                w = i.split()
                self.pos_word_count += len(w)
            else:
                w = i.split()
                self.neg_word_count += len(w)
        print("Word count in pos: ",self.pos_word_count)
        print("Word count in neg: ",self.neg_word_count)
        

    def calculate_class_counts(self):
        # Count the occurrences of 'pos' and 'neg' tags
        for label in self.y:
            if label == self.pos_tag:
                self.pos_count += 1
            else:
                self.neg_count += 1
        print("Count of pos tag: ",self.pos_count)
        print("Count of neg tag: ",self.neg_count)
        

    def find_word_pos(self, word):
        # Find the probability of a word given class 'pos'
        count = 0
        for i, j in zip(self.x, self.y):
            w = i.split()
            if j == self.pos_tag and word in w:
                count += 1

        print(f"P(word='{word}'|pos): ",(count + 1) / (self.pos_word_count + self.tot))
        return (count + 1) / (self.pos_word_count + self.tot)  # Laplace smoothing
        
        

    def find_word_neg(self, word):
        # Find the probability of a word given class 'NS'
        count = 0
        for i, j in zip(self.x, self.y):
            w = i.split()
            if j == self.neg_tag and word in w:
                count += 1
                
        print(f"P(word='{word}'|neg): ",(count + 1) / (self.neg_word_count + self.tot))
        return (count + 1) / (self.neg_word_count + self.tot)  # Laplace smoothing
        
   

    def classify(self, sentence):
        sent = sentence.split()
        
        # Calculate P(word|pos) and P(word|neg)
        p_word_pos = [self.find_word_pos(i) for i in sent]
        p_word_neg = [self.find_word_neg(i) for i in sent]
        print("P(word|pos): ",p_word_pos)
        print("P(word|neg): ",p_word_neg)
        
        
        # Prior probabilities
        p_pos = self.pos_count / len(self.y)
        p_neg = self.neg_count / len(self.y)
        
        # Calculate P(sentence|pos) and P(sentence|neg)
        p_text_pos = 1
        for prob in p_word_pos:
            p_text_pos *= prob
        p_pos_text = p_text_pos * p_pos
        print(p_pos_text)
        
        p_text_neg = 1
        for prob in p_word_neg:
            p_text_neg *= prob
        p_neg_text = p_text_neg * p_neg
        print(p_neg_text)
        
        
        # Classification decision
        if p_pos_text >= p_neg_text:
            return "Positive"
        else:
            return "Negative"

    def evaluate(self):
        # Initialize counters for TP, FP, FN, and TN
        tp, fp, fn, tn = 0, 0, 0, 0
        predictions = []

        # Predict for each entry in the dataset and compare with actual label
        for i, actual in enumerate(self.y):
            # predicted = self.classify(self.x[i])
            # predictions.append(predicted)
            sentence = self.df['Text Documents'].iloc[i]  # Correct way to access the sentence
            predicted = self.classify(sentence)

            # True Positive (TP): Correctly predicted Sports
            if actual == self.pos_tag and predicted == self.pos_tag:
                tp += 1

            # True Negative (TN): Correctly predicted Not Sports
            elif actual == self.neg_tag and predicted == self.neg_tag:
                tn += 1

            # False Positive (FP): Incorrectly predicted Sports
            elif actual == self.neg_tag and predicted == self.pos_tag:
                fp += 1

            # False Negative (FN): Incorrectly predicted Not Sports
            elif actual == self.pos_tag and predicted == self.neg_tag:
                fn += 1

        # Calculate accuracy, precision, recall
        accuracy = (tp + tn) / len(self.y)
        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall
        }
# Example usage:
# Assuming 'df' is a pandas DataFrame with columns 'text' and 'tag'
# df = pd.DataFrame({
#     'text': ['A great game', 'The election was over', 'Very clean match', 'A clean but forgettable game', 'It was a close election'],
#     'tag': ['S', 'NS', 'S', 'S', 'NS']
# })
df = pd.read_csv('TextClassifier2.csv')

# Instantiate the classifier and classify a sentence
classifier = NaiveBayesTextClassifier(df)
sentence = 'great restaurant'
classification = classifier.classify(sentence)

print(f'The sentence "{sentence}" is classified as: {classification}')

metrics = classifier.evaluate()
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")


Word count in pos:  44
Word count in neg:  57
Count of pos tag:  9
Count of neg tag:  9
P(word='great'|pos):  0.019417475728155338
P(word='restaurant'|pos):  0.009708737864077669
P(word='great'|neg):  0.008620689655172414
P(word='restaurant'|neg):  0.017241379310344827
P(word|pos):  [0.019417475728155338, 0.009708737864077669]
P(word|neg):  [0.008620689655172414, 0.017241379310344827]
9.425959091337542e-05
7.431629013079666e-05
The sentence "great restaurant" is classified as: Positive
P(word='i'|pos):  0.038834951456310676
P(word='love'|pos):  0.02912621359223301
P(word='this'|pos):  0.04854368932038835
P(word='sandwich'|pos):  0.019417475728155338
P(word='i'|neg):  0.0603448275862069
P(word='love'|neg):  0.008620689655172414
P(word='this'|neg):  0.05172413793103448
P(word='sandwich'|neg):  0.008620689655172414
P(word|pos):  [0.038834951456310676, 0.02912621359223301, 0.04854368932038835, 0.019417475728155338]
P(word|neg):  [0.0603448275862069, 0.008620689655172414, 0.0517241379310344